In [1]:
from preprocessing_functions import *
import networkx as nx
from tqdm import tqdm
import numpy as np
import pandas as pd

# Functions

In [7]:
def BFS(Graph, Source, adj_list):
    
    '''
    In this function we implemented the breadth first search algorithm in order to evaluate the distance, 
    from a source node, of all the node in the graph.
    If a node is not linked to any other nodes it dosn't appeare in the dictionary called distance.
    '''
    
    # initialization
    distance = dict()
    visited = {k: False for k in Graph.nodes}
    queue = []

    queue.append(Source)
    distance[Source] = 0
    visited[Source] = True

    while queue:
        v = queue.pop()
        
        # check if the node has a list of first neighbors
        if v in adj_list.keys(): 
            
            # evaluate the distance and add new nodes to the queue
            for node in adj_list[v]:
                if visited[node] == False:
                    visited[node] = True
                    distance[node] = distance[v] + 1
                    queue.append(node)

    return distance

In [3]:
def Adjacency_list(Graph):
    
    '''
    In this function returnt the adjacency list, a dictionary in which to every node is associated
    an array of first neighbors nodes. It's take in imput a networkx graph object.
    '''
    
    List = dict()
    
    for source, destination in Graph.edges:
        
        if source not in List.keys():
            List[source] = []
        List[source].append(destination)    
    
    return List    

In [4]:
def Class_Distance(Graph, C0, categories):
    
    '''
    This function evaluate the distance between two class evaluating the mean of the shortest path 
    for each pair of the nodes in the sub graph cosisting of the nodes belonging to two different class.
    If the sets of nodes belonging to two different class are not linked by any edge, 
    we consuder the distance between infinite.
    '''
    
    Class_distance = dict()
    
    edges_C0 = Graph.subgraph(categories[C0]).edges
    
    for C in tqdm(category_dic.keys()):
        
        if C != C0:
            
            edges_C = Graph.subgraph(categories[C]).edges
            sub_Graph = Graph.subgraph(categories[C0] + categories[C])

            connected = []
            
            #check if there is at least one link between sets of nodes belonging to two different class
            for edge in sub_Graph.edges():
                if edge not in edges_C0 and edge not in edges_C:
                    connected.append(True)
            
            # evaluate shortest path between each pair using using repeated breadth first search
            if connected:
                distance = []
                adj_list = Adjacency_list(sub_Graph)

                for node in adj_list.keys():
                    d = BFS(sub_Graph, node, adj_list)
                    distance += [d[k] for k in d.keys() if d[k]!= 0]

                Class_distance[C] = np.mean(distance)

            else:
                Class_distance[C] = float('inf')
                
    return Class_distance

# Preprocessing

In [5]:
inverted_dic = Build_inverted_dictionary('wiki-topcats-categories.txt')

category_dic = dict()
for k in inverted_dic.keys():
    if inverted_dic[k] not in category_dic.keys():
        category_dic[inverted_dic[k]] = list()
    category_dic[inverted_dic[k]].append(k)
    
Nodes = list(inverted_dic.keys())

DG = Build_graph('wikigraph_reduced.csv', Nodes)

# RQ 5

In [8]:
distance_class = Class_Distance(DG, 'American_television_actors', category_dic)

100%|████████████████████████████████████████████████████████████████████████████| 17357/17357 [12:54<00:00, 22.40it/s]


In [26]:
pd.DataFrame.from_dict(distance_class, orient='index', columns=['distance'])\
.sort_values('distance', ascending=True).head(60)

,distance
Year_of_birth_missing,2.018668
Indian_films,2.059304
Hindi-language_films,2.120028
Major_League_Baseball_pitchers,2.140373
British_films,2.141878
"People_from_Los_Angeles,_California",2.147941
African_American_films,2.150675
American_science_fiction_horror_films,2.151424
1986_films,2.152466
1930s_comedy_films,2.154081


In [30]:
distance_class_1 = Class_Distance(DG, 'Rivers_of_Cara-Severin_County', category_dic)  

100%|███████████████████████████████████████████████████████████████████████████| 17357/17357 [00:19<00:00, 896.51it/s]


In [32]:
pd.DataFrame.from_dict(distance_class_1, orient='index', columns=['distance'])\
.sort_values('distance', ascending=True).head(15)

,distance
Rivers_of_the_Mure_subbasin,2.082474
Rivers_of_the_Timi-Bega_subbasin,2.180222
International_rivers_of_Europe,2.190476
Rivers_of_Hunedoara_County,2.260914
Rivers_of_Romania,2.344426
Rivers_of_Gorj_County,2.350078
Rivers_of_the_Nera-Cerna_subbasin,2.697004
Buprestoidea,inf
Skyscrapers_between_100_and_149_meters,inf
"Skidmore,_Owings_and_Merrill_buildings",inf
